# Nuoseklių sakinių klasifikavimas atsitiktinių imčių medicininių tyrimų santraukose (galutinis modelis)

Dokumente "[**Neural Networks for Joint Sentence Classification in Medical Paper Abstracts**](https://arxiv.org/pdf/1612.05251)" sudarytas ANN modelis pasiekė 89.9% tikslumą po mokymo su visais **PubMed 20k RCT** rinkinio pavyzdžiais. Mūsų geriausias Modelis 5, mokytas su 10% **PubMed 20k RCT** pavyzdžių, santraukų sakinių klasifikaciją atlieka ~85.6% tikslumu.

Šiame etape sudarysim NSK Modelį, kuris bus eksperimentų metu sudarytas Modelis 5 apmokytas su visais **PubMed 20k RCT** rinkinio pavyzdžiais, mokymą atliekant daugiau nei 5 epochas.

In [11]:
# bibliotekų importavimas
import os
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import tensorflow as tf
from tensorflow.keras import layers

In [3]:
# išjungiam TensorFlow įspėjimų (warnings) vaizdavimą rezultatuose
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

# glabalios TensorFlow random seed reikšmės nustatymas
tf.random.set_seed(42)

In [4]:
# pagalbinių funkcijų importavimas
from helper_functions.dataset_functions import *

In [7]:
# parametrai gauti eksperimentavimo metu

# santraukos sakinio išskirstyto žodžiais vektoriaus dydis
output_sentence_len = 55

# santraukos sakinio išskirstyto simboliais vektoriaus dydis
output_sentence_chars_len = 290

# simbolių embedding'o vetoriaus dydis
max_char_tokens = 28

# dažniausiai psikartojančių sakinių numerių dydis
line_numbers_depth = 17

# didžiausia dalį santraukų sudarančių sakinių sakičius
sentences_total_depth = 22

## Duomenų paruošimas

### Pandas DataFrame sudarymas

In [13]:
# duomenų rinkinio tekstinių failų nuskaitymas
DATASET_PATH = "pubmed-rct/PubMed_20k_RCT_numbers_replaced_with_at_sign/"

_, train_dicts = convert_file_lines_to_dic(DATASET_PATH + 'train.txt')
_, test_dicts = convert_file_lines_to_dic(DATASET_PATH + 'test.txt')
_, val_dicts = convert_file_lines_to_dic(DATASET_PATH + 'dev.txt')

In [14]:
# pandas DataFrame sudarymas
train_df = pd.DataFrame(train_dicts)
test_df = pd.DataFrame(test_dicts)
val_df = pd.DataFrame(val_dicts)

In [15]:
# sakinių sąrašų sudarymas
X_train = train_df["text"].to_list()
X_test = test_df["text"].to_list()
X_val = val_df["text"].to_list()

In [26]:
# sakinių išskaidytų simboliais sąrašas
X_train_chars = [split_sentence_chars(sentence) for sentence in X_train]
X_test_chars = [split_sentence_chars(sentence) for sentence in X_test]
X_val_chars = [split_sentence_chars(sentence) for sentence in X_val]

### Duomenų formato pakeitimas (encoding)

In [16]:
# sakinių etikėčių kodavimas su LabelEncoder
label_encoder = LabelEncoder()

y_train_label = label_encoder.fit_transform(train_df["target"])
y_test_label = label_encoder.transform(test_df["target"])   
y_val_label = label_encoder.transform(val_df["target"])

etiketes = label_encoder.classes_

In [17]:
# sakinių etikėčių kodavimas TensorFlow modeliui su OneHotEncoder
onehot_encoder = OneHotEncoder(sparse_output=False)

y_train_onehot = onehot_encoder.fit_transform(train_df["target"].to_numpy().reshape(-1, 1))
y_test_onehot = onehot_encoder.transform(test_df["target"].to_numpy().reshape(-1, 1))   
y_val_onehot = onehot_encoder.transform(val_df["target"].to_numpy().reshape(-1, 1))

In [19]:
# sakinių numerių kodavimas su TensorFlow OneHot nurodant gylį
train_line_numbers_onehot = tf.one_hot(train_df["number"].to_numpy(), depth=line_numbers_depth)
test_line_numbers_onehot = tf.one_hot(test_df["number"].to_numpy(), depth=line_numbers_depth)
val_line_numbers_onehot = tf.one_hot(val_df["number"].to_numpy(), depth=line_numbers_depth)

In [20]:
# santraukų sakinių skaičiaus kodavimas su TensorFlow OneHot nurodant gylį
train_sentences_total_onehot = tf.one_hot(train_df["total"].to_numpy(), depth=sentences_total_depth)
test_sentences_total_onehot = tf.one_hot(test_df["total"].to_numpy(), depth=sentences_total_depth)
val_sentences_total_onehot = tf.one_hot(val_df["total"].to_numpy(), depth=sentences_total_depth)

### TensorFlow Dataset sudarymas

In [27]:
# training Dataset sudarymas
train_data = tf.data.Dataset.from_tensor_slices((train_line_numbers_onehot,
                                                 train_sentences_total_onehot,
                                                 X_train,
                                                 X_train_chars))

train_labels = tf.data.Dataset.from_tensor_slices(y_train_onehot)

train_dataset = tf.data.Dataset.zip((train_data, train_labels)).batch(32).prefetch(tf.data.AUTOTUNE)

In [28]:
# testing Dataset sudarymas
test_data = tf.data.Dataset.from_tensor_slices((test_line_numbers_onehot,
                                                test_sentences_total_onehot,
                                                X_test,
                                                X_test_chars))

test_labels = tf.data.Dataset.from_tensor_slices(y_test_onehot)

test_dataset = tf.data.Dataset.zip((test_data, test_labels)).batch(32).prefetch(tf.data.AUTOTUNE)

In [29]:
# validation TensorFlow Dataset sudarymas
val_data = tf.data.Dataset.from_tensor_slices((val_line_numbers_onehot,
                                               val_sentences_total_onehot,
                                               X_val,
                                               X_val_chars))

val_labels = tf.data.Dataset.from_tensor_slices(y_val_onehot)

val_dataset = tf.data.Dataset.zip((val_data, val_labels)).batch(32).prefetch(tf.data.AUTOTUNE)

## Modelio sudarymas

### Žodžių vektorizacijos sluoksnio paruošimas (Universal Sentence Encoder)

In [18]:
# importuojam TensorFlow Hub modulį
import tensorflow_hub as hub

# atsisiunčiam if TF Hub Universal Sentence Encoder
word_tokens_embedder_layer = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder/4", # nuoroda į iš anksto apmokyta modelį
                                            trainable=False, # nurodome nekeisti modelio išmoktų savybių
                                            name="universal_sentence_encoder")

2023-03-21 11:09:28.763763: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-21 11:09:29.276377: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 4653 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1060 6GB, pci bus id: 0000:01:00.0, compute capability: 6.1


### Simbolių vektorizacijos sluoksnio paruošimas

In [30]:
# sukuriam vektorizavimo sluoksnį
char_vectorizer = layers.TextVectorization(max_tokens=max_char_tokens,
                                           output_sequence_length=output_sentence_chars_len,
                                           name="char_vectorizer")

# sukuriam tokenizacijos žodyną naudojant į simbolius išskaidytus X_train_chars sakinius
char_vectorizer.adapt(X_train_chars)

In [31]:
# tokenizacijos metu sudaryto žodyno apžvalga
char_tokens_vocabulary = char_vectorizer.get_vocabulary()

In [32]:
#  sukuriam simbolių embedding'o sluoksnį
char_tokens_embedder_layer = layers.Embedding(input_dim=len(char_tokens_vocabulary),
                                              output_dim=24, # rezultato vektoriaus dydis
                                              mask_zero=True, # paslepiam vektorizuoto sakinio tuščias reikšmes
                                              name="char_tokens_embedding")

## Modelio mokymas

In [ ]:
# TensorBoard callback funkcijos sudarymas
tensorboard_callback = tf.keras.callbacks.TensorBoard(
    log_dir="tensorboard",  # aplankas kur saugoti log failus
)

In [ ]:
# ModelCheckpoint callback funkcijos sudarymas
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath="checkpoints/m6.ckpt", # failo, kuriame saugosime atskaitos tašką pavadinimas
    monitor="val_accuracy",         # stebimas parametras, kurio atžvilgiu kursime taškus (įvertinimo tikslumas)
    verbose=0,                      # išjungiam funkcijos pranešimus
    save_best_only=True,            # nurodom saugoti tik modelį esant geriausiam stebimam parametrui
    save_weights_only=True,         # saugojam tik įverčius, bet ne visą modelį
    save_freq='epoch',              # kaip dažnai vertinti stebimą parametrą
)

In [ ]:
# EarlyStopping callback fnkcijos sudarymas
early_stopping_callback = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",         # parametras, kurio pokyčiai bus stebimi
    min_delta=0.001,            # dydis nurodantis koks mažiausias stebimo parametro teigiamas pokytis yra priimtinas
    patience=5,                 # kiek epochų laukti stebimo parametro pokyčio, prieš sustabdant mokymą
    verbose=0,                  # išjungiam funkcijos pranešimus
    restore_best_weights=True,  # atstatyti modelio stadiją su geriausiasiais mokymosi rezultatais
)

In [ ]:
# Modelio 5 klonavimas
model_6 = tf.keras.models.clone_model(model_5)
model_6.set_weights(model_5.get_weights())
model_6._name = "model_6"

In [ ]:
# išvedam sudaryto modelio struktūrą
model_6.summary()

In [ ]:
# sukūriam (kompiliuojam) aprašytą modelį
model_6.compile(loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.1),
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

In [ ]:
# sudarom modelio "Modelis 6" vertinimo diagramas
plot_accuracy_loss_curves(model_6_history, "6")

In [ ]:
# įvertinam modelį su visais testavimo rinkinio pavyzdžiais
# su visais training 20k pavyzdziu apmokytas modelis evaluatina 85.15%
loss, accuracy = model_6.evaluate(test_tokens_chars_pos_dataset)
print(f"Modelio 6 tikslumas su testavimo duomenimis: {(accuracy*100):.2f}%")